<div style="color:White; display:fill; border-radius:5px;background-color:#336b87;font-size:270%;font-family:sans-serif;letter-spacing:0.5px:text-align: center">
Tabular Playground Series - Oct 2021
</div>

<a><img src="https://i.ibb.co/PWvpT9F/header.png" alt="header" border="0" ></a>

In [ ]:
import pandas as pd, numpy as np, os, matplotlib.pyplot as plt, seaborn as sns
import datatable as dt
import warnings
import random
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings('ignore')
import gc
from sklearn.metrics import roc_curve,auc
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
import plotly.figure_factory as ff
import plotly.express as px
from xgboost import XGBClassifier 
pd.set_option('max_columns',None)

from time import time
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
PLOT = True

#notebook setup
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

    
TARGET = 'target'
FOLD = 5
SEED = 69
N_ESTIMATORS=10000
DEVICE = 'CPU'
EVAL_METRIC = "AUC"

STUDY_TIME = 60*60*8
seed_everything(SEED)

<div style="color:White; display:fill; border-radius:5px;background-color:#dd4124;font-size:170%;font-family:sans-serif;letter-spacing:0.5px">
Note: Upvote is Free!!!
</div>
<div style="color:White; display:fill; border-radius:5px;background-color:#009473;font-size:111%;font-family:sans-serif;letter-spacing:0.5px">
I will public my dataset after you upvote it, probably this will be my last submission, bcoz of my exam. :(
</div>

In [ ]:
%time
#import data
train = dt.fread(r"../input/tps-october-2021-dataset/train_1.csv").to_pandas()
test = dt.fread(r"../input/tps-october-2021-dataset/test_1.csv").to_pandas()

train.columns = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7',
       'f9', 'f10', 'f11', 'f12', 'target']

test.columns = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7',
       'f9', 'f10', 'f11', 'f12']

train['target'] = train['target'].astype('int64') 

sample_submission = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')

In [ ]:
print('Train Shape: ',train.shape)
train.tail(10).reset_index(drop=True)

In [ ]:
print('Test Shape: ',test.shape)
test.head(10)

<div style="color:White; display:fill; border-radius:5px;background-color:#336b87;font-size:170%;font-family:sans-serif;letter-spacing:0.5px">
Preprocessing
</div>

In [ ]:
#setup for preprocessing
X = train.drop(TARGET, axis=1)
y = train[[TARGET]]
X_test = test

del train, test
gc.collect()

In [ ]:
cols = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f10',
       'f12']

X = X[cols]
X_test = X_test[cols] 

In [ ]:
# helper functions
def get_auc(y_true, y_hat):
    fpr, tpr, _ = roc_curve(y_true, y_hat)
    score = auc(fpr, tpr)
    return score

<div style="color:White; display:fill; border-radius:5px;background-color:#336b87;font-size:170%;font-family:sans-serif;letter-spacing:0.5px">
Train-set KDE Plot
</div>

In [ ]:
if PLOT == True:
    X_data = [X.f0,  X.f1,  X.f2,  X.f3,  X.f4,  X.f5,  X.f6,  X.f10,  X.f12]
    group_labels = X.columns.to_list()
    fig = ff.create_distplot(X_data, group_labels, bin_size=0.3, show_hist=False, show_rug=False)
    fig.show()

<div style="color:White; display:fill; border-radius:5px;background-color:#336b87;font-size:170%;font-family:sans-serif;letter-spacing:0.5px;text-align: center">
Test-set KDE Plot
</div>

In [ ]:
if PLOT == True:
    _data = [X_test.f0,  X_test.f1,  X_test.f2,  X_test.f3,  X_test.f4,  X_test.f5,  X_test.f6,  X_test.f10,  X_test.f12]
    fig = ff.create_distplot(_data, group_labels, bin_size=0.3, show_hist=False, show_rug=False)
    fig.show()

<div style="color:White; display:fill; border-radius:5px;background-color:#336b87;font-size:170%;font-family:sans-serif;letter-spacing:0.5px;text-align: center">
Train-set Heatmap Plot
</div>

In [ ]:
if PLOT == True:
    #correlation between all models pred
    data = np.corrcoef(X_data)
    fig=px.imshow(data,x=group_labels, y=group_labels)

    fig.show()

<div style="color:White; display:fill; border-radius:5px;background-color:#336b87;font-size:170%;font-family:sans-serif;letter-spacing:0.5px;text-align: center">
Test-set Heatmap Plot
</div>

In [ ]:
if PLOT == True:
    #correlation between all models pred
    data = np.corrcoef(_data)
    fig=px.imshow(data,x=group_labels, y=group_labels)

    fig.show()

<div style="color:White; display:fill; border-radius:5px;background-color:#336b87;font-size:170%;font-family:sans-serif;letter-spacing:0.5px;text-align: center">
Simple LightGBM
</div>

In [ ]:
params = {'device_type': 'cpu',
 'n_estimators': 10000,
 'learning_rate': 0.10932948761384702,
 'num_leaves': 780,
 'max_depth': 7,
 'min_data_in_leaf': 9200,
 'lambda_l1': 20,
 'lambda_l2': 5,
 'min_gain_to_split': 8.857618869765401,
 'bagging_fraction': 0.8,
 'bagging_freq': 1,
 'feature_fraction': 0.8}

overfit = {'device': 'cpu',
#               'gpu_platform_id': 0,
#               'gpu_device_id': 0,
              'num_threads': -1,
         'objective': 'binary','metric' : 'auc',
    'n_estimators': 100000,
    'random_state': SEED,
    'learning_rate': 3e-3,
         }

In [ ]:
#check of all the columns in train is in test set
assert X.columns.to_list() == X_test.columns.to_list()

meta_pred_tmp = []
scores_tmp = []

# create cv
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

for fold, (idx_train, idx_valid) in enumerate(kf.split(X, y)):
    # create train, validation sets
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
    model = LGBMClassifier(**overfit)
#     model = LogisticRegression(**{'C': 0.0016915132862548748, 'max_iter': 29, 'solver': 'lbfgs'})
#     model = RandomForestClassifier(random_state=SEED)
    model.fit(X_train, y_train
              ,eval_set=[(X_valid,y_valid)],
             early_stopping_rounds=200,
            verbose=0
             )
    # validation prediction
    pred_valid = model.predict_proba(X_valid)[:,1]
    
    score = get_auc(y_valid, pred_valid)
    scores_tmp.append(score)
    
    print(f"Fold: {fold + 1} Score: {score}")
    print('--'*20)
    
    # test prediction based on oof_set
    y_hat = model.predict_proba(X_test)[:,1]
    meta_pred_tmp.append(y_hat)
# print overall validation scores
print(f"Overall Validation Score | Meta: {np.mean(scores_tmp)}")
print('::'*20)

In [ ]:
#average meta predictions over each fold
meta_predictions = np.mean(np.column_stack(meta_pred_tmp), axis=1)

# create submission file
stacked_submission = sample_submission.copy()
stacked_submission[TARGET] = meta_predictions
stacked_submission.to_csv('./LGBM_overfit.csv', index=False)

<div style="color:White; display:fill; border-radius:5px;background-color:#336b87;font-size:170%;font-family:sans-serif;letter-spacing:0.5px;text-align: center">
LightGBM Prediction KDE Plot
</div>

In [ ]:
if PLOT == True:
    plot = pd.concat([X_test,stacked_submission[TARGET]],axis=1)

    pred = [plot.f0,  plot.f1,  plot.f2,  plot.f3,  plot.f4,  plot.f5,  plot.f6,  plot.f10,  plot.f12, plot.target]
    group_labels = plot.columns.to_list()
    fig = ff.create_distplot(pred, group_labels, bin_size=0.3, show_hist=False, show_rug=False)
    fig.show()